In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

## Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

engine


In [ ]:
# reflect an existing database into a new model
# reflect the tables
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()


In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station


In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

## Inspect the hawaii data base

In [ ]:
from sqlalchemy import inspect
inspector = inspect(engine)

columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
from sqlalchemy import inspect
inspector = inspect(engine)

columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
latest = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
latest

In [ ]:
# Perform a query to retrieve the data and precipitation scores

precip_last_year = session.query(Measurement.date,Measurement.prcp).\
                    filter(func.strftime("%Y-%m-%d", Measurement.date) >= '2016-08-23').\
                    order_by(Measurement.date).all()
precip_last_year

precip_df = pd.DataFrame(precip_last_year, columns=['Date', 'Precip'])

precip_df = precip_df.dropna()
#precip_df.head()
#precip_df.set_index("Date", inplace = True)
#precip_df.head(20)

#precip_df['Date'] = pd.to_datetime(precip_df['Date'])
precip_df = precip_df.set_index('Date')

#precip_df['Precip'].count()
precip_df.head()


In [ ]:
df = precip_df
labels = df.index

# Set x axis and tick locations
x_axis = np.arange(len(labels))
tick_locations = [value+0.2 for value in x_axis]

#set fig size, x-axis, y
plt.figure(figsize=(20,8))
width = 3

#plot  data
plt.bar(x_axis - width, df['Precip'],width = width, color='g', alpha=0.5, align="edge", label = 'precip(in)')
#plt.xticks(tick_locations, df.index, rotation="vertical", fontsize=16)
major_ticks = np.arange(0,2021,100)
plt.xticks(major_ticks, df.index, rotation="vertical", fontsize = 10)

# Set x and y limits
plt.xlim(-0.25, len(x_axis))
plt.ylim(0, max(df["Precip"]))

# Set Title and labels
plt.title("Hawaii Precipitation from 8/16-8/17 ", fontdict = {'fontsize' : 25})
plt.xlabel("Date", fontdict = {'fontsize' : 5})
plt.ylabel("precip",  fontdict = {'fontsize' : 18})
plt.savefig("Images/precipitation.png")
plt.show()




In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?

stations = session.query(Station.name).count()

print(f"the are {stations} stations")



In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
# List the stations and the counts in descending order.
active_list = session.query(Measurement.station, func.count(Measurement.station)).\
            group_by(Measurement.station).\
            order_by(func.count(Measurement.station).desc()).all()

active_list

most_active = active_list[0][0]
most_active



In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, # highest temperature recorded, 
#and average temperature of the most active station?

low = session.query(func.min(Measurement.tobs).filter(Measurement.station == most_active)).all()[0][0]
avg = round(session.query(func.avg(Measurement.tobs).filter(Measurement.station == most_active)).all()[0][0],2)
hi = session.query(func.max(Measurement.tobs).filter(Measurement.station == most_active)).all()[0][0]

print (f"low temp = {low}")
print (f"avg temp = {avg}")
print (f"high temp = {hi}")






In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

# Calculate the date 1 year ago from the last data point in the database
latest = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(latest)

twelve_months_ago = dt.datetime.strptime(latest[0], '%Y-%m-%d') - dt.timedelta(days=365)
twelve_months_ago

last_twelve_temps = session.query(Measurement.station, Measurement.tobs).\
                filter(Measurement.station == most_active).\
                filter(Measurement.date >= twelve_months_ago).all()


#make pd_df for plotting
temps_df = pd.DataFrame(last_twelve_temps)

temps_df.head(20)





In [ ]:
station_name = session.query(Station.name).filter(Station.station == "USC00519281").all()
station_name

#last_twelve_temps = session.query(Measurement.station, Measurement.tobs).\
#                filter(Measurement.station == most_active).\
#                filter(Measurement.date >= twelve_months_ago).all()

In [ ]:
active_station = session.query(Station.name).filter(Station.station == most_active).all()[0][0]
active_station


In [ ]:
# Plot the results as a histogram with bins=12.
temps_df.plot.hist(by='station', bins=12, figsize=(12,8), color="green")
plt.title("Frequency of temperatures measurements for Station " + active_station, fontsize=16)
plt.xlabel("Temperature", fontsize=16)
plt.savefig("Images/station_temps.png")
plt.show()